In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException


site_ec_dict = {'google': EC.element_to_be_clickable((By.XPATH, "//div[@class='QS5gu sy4vM']")),
                'nasdaq': EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))}

def accept_cookies(site = 'google'):    
    try:
        accept = WebDriverWait(driver, 10).until(site_ec_dict[site])
        accept.click()
    except:
        print("No 'Accept' button found or unable to click.")


def google(query):
    # Open the website
    driver.get("https://google.com")
    accept_cookies('google')
    search_box = driver.find_element(By.NAME, "q")

     # Enter your search query
    search_query = query
    search_box.send_keys(search_query)

    # Simulate hitting Enter
    search_box.send_keys(Keys.RETURN)



def click_site(site):
    driver.find_element(By.XPATH, f"//a[contains(@href, '{site}')]").click()







In [3]:
driver = webdriver.Chrome()

google('nasdaq earnings calendar')
click_site('nasdaq.com')
accept_cookies('nasdaq')

